In [1]:
import os
import PIL
import time
import math
import copy
import torch
import random
import glob as gb
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import resnet50_128_redesign as model
from DataProcessing import process_dataloder
from PIL import Image
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [2]:
dataloaders, class_names, dataset_sizes,expA = process_dataloder()
ite = len(dataloaders["train"])
print(dataset_sizes)

train:   0%|          | 5/10448 [00:00<03:34, 48.78it/s]

----------
folders for train and test:  245 83
----------
data for train and test:
['negative', 'positive', 'surprise', 'non_micro-expression']
train [2168, 1856, 1408, 5016]
test [832, 600, 576, 1816]
dataset_size: {'train': [0, 10448], 'test': [10448, 14272]}
---------- load data ----------


test:   1%|          | 39/3824 [00:00<00:09, 385.58it/s]

---------- load data ----------


test: 100%|██████████| 3824/3824 [00:20<00:00, 188.97it/s]

Data loading complete in 1m 18s
{'train': 43272, 'test': 16000}


In [3]:
print(ite)

1353


In [4]:
def training(model, criterion, optimizer, scheduler, lr_find_epochs = 2):
    running_loss = 0.0
    running_corrects = 0
    lr_find_loss = []
    lr_find_lr = []
    
    iter1 = 0
    smoothing = 0.05
    for i in range(lr_find_epochs):
        print('Epoch {}/{}'.format(i, lr_find_epochs - 1))
        print('-' * 10)
        # Iterate over data.        
        for inputs, labels in dataloaders["train"]:   

            # Send to device
            inputs = inputs.to(device)            
            labels = labels.to(device)

            # Training mode and zero gradients
            model.train()
            optimizer.zero_grad()

            # Get outputs to calc loss
            outputs = model(inputs)                       
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
#             print("loss1:", loss)
            # Backward pass
            loss.backward()
            optimizer.step()                  

            # Update LR
            scheduler.step()
            lr_step = optimizer.state_dict()["param_groups"][0]["lr"]
            lr_find_lr.append(lr_step)
            
            # smooth the loss
            if iter1==0:
                lr_find_loss.append(loss)
            else:
                loss = smoothing * loss + (1 - smoothing) * lr_find_loss[-1]
                lr_find_loss.append(loss)
#                 print("loss2:", loss)
            iter1 += 1

    return model, lr_find_loss, lr_find_lr   

In [10]:
#download the pre-trained model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
features_layers = 91
#Freeze layers before classifier
def freezing(features_layers,model):   
    lay_mark = 0;
    para_list = []
    for param in model.parameters():
        if lay_mark > features_layers:
            para_list.append(param)
        if lay_mark <= features_layers:
            param.requires_grad = False
        lay_mark += 1
    return para_list,model

def LR_ini(model,ite):
    #Freeze layers before classifier
    para_list, model = freezing(features_layers,model)

    #define lower boundary and upper boundary    
    lr_find_epochs = 1
    start_lr = 1e-7
    end_lr = 4*1e-2
    optimizer = torch.optim.SGD(para_list, start_lr)    
    #Optimiazation retraining layers
    criterion = nn.CrossEntropyLoss()
    # LR function lambda
    lr_lambda = lambda x: math.exp(x * math.log(end_lr / start_lr) / (lr_find_epochs * ite))
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
    
    return criterion, optimizer, scheduler, lr_find_epochs

In [6]:
#step and learning rate 
def lr_step(lr_find_lr,name):  
    
    step = [i for i in range(len(lr_find_lr))]
    fig = go.Figure(data=go.Scatter(x=step, y=lr_find_lr))
    fig.update_layout(title='lr-step plot',
                   xaxis_title='step',
                   xaxis_type="log",
                   yaxis_title='learning rate')
    fig.show()
    
# loss-lr plot 
def loss_lr(lr_find_lr, lr_find_loss,name):
    
    fig = go.Figure(data=go.Scatter(x=lr_find_lr, y=torch.tensor(lr_find_loss)))
    fig.update_layout(title='loss-lr plot',
                      xaxis_type="log",
                   xaxis_title='learning rate',
                   yaxis_title='loss')
#     fig.write_image('fig/'+ name)
    fig.show()

In [11]:
modelRes50 = model.resnet50_128(weights_path='./model/resnet50_128.pth')
modelRes50.add_module("feat_extract1",nn.Conv2d(128, 64, kernel_size=[1, 1], stride=(1, 1), bias=False))
modelRes50.add_module("feat_extract2",nn.Conv2d(64, 4, kernel_size=[1, 1], stride=(1, 1), bias=False))
modelRes50.add_layers([modelRes50.feat_extract1, modelRes50.feat_extract2])
modelRes50 = modelRes50.to(device)

criterion, optimizer, scheduler, lr_find_epochs = LR_ini(modelRes50,ite)

#run network
modelRes50, lr_find_loss, lr_find_lr = training(modelRes50, criterion, optimizer, scheduler, lr_find_epochs)

#plot graph
lr_step(lr_find_lr,'StaSpeLr-step.jpg')
loss_lr(lr_find_lr, lr_find_loss,'StaSpeLoss-Lr.jpg')

Epoch 0/0
----------
